In [7]:
with open("inputs/Day_15.txt") as f:
    raw_input_data = f.read()

In [80]:
from collections import namedtuple, defaultdict
from enum import Enum


class Block(Enum):
    Wall = 0
    Open_Passage = 1
    Oxygen_System = 2
    
class Move(Enum):
    Up = 1 # North
    Down = 2 # South
    Left = 3 # West
    Right = 4 # East 

    
def part_1_solution(raw_input):
    puzzle_input = list(map(int, raw_input.split(',')))
    
    robot = Robot(puzzle_input)
    
    grid = build_grid(robot)
    
    path_to_oxygen_system = grid.get_path_to(grid.blocks[Block.Oxygen_System], (0, 0))
    
    return len(path_to_oxygen_system) - 1 # without transition to start position

def build_grid(robot):
    grid = Grid()
    
    def dfs(position):
        for neighbor in get_all_neighbors(position):
            if neighbor not in grid.blocks:
                
                neighbor_block = robot.explore_block(grid, neighbor)
                if neighbor_block != Block.Wall:
                    dfs(neighbor)
                
                # return to intitial position
                if robot.position != position:
                    robot.move_to(grid, position)
    
    dfs((0, 0))
    
    return grid

def get_all_neighbors(position):
    for x_offset, y_offset in ((0, 1), (1, 0), (0, -1), (-1, 0)):
        canditdate_position = (position[0] + x_offset, position[1] + y_offset)

        yield canditdate_position

        
class Robot:
    def __init__(self, program):
        self.computer = start_computer(program)
        next(self.computer)
        self.position = (0, 0)
        
    def explore_block(self, grid, target_position):
        move_command = self._get_movement_command(target_position)
        
        raw_response = self.computer.send(move_command)
        next(self.computer)
        
        target_block = Block(raw_response.status)
        
        grid.blocks[target_position] = target_block
        
        if target_block != Block.Wall:
            self.position = target_position
            
        if target_block == Block.Oxygen_System:
            grid.blocks[Block.Oxygen_System] = target_position
        
        return target_block
    
    def move_to(self, grid, target_position):
        path = grid.get_path_to(target_position, self.position)

        while path:
            next_position = path.pop(0)
            
            if next_position == self.position:
                continue
            
            move_command = self._get_movement_command(next_position)
            self.computer.send(move_command)
            self.position = next_position
            next(self.computer)
    
    
    def _get_movement_command(self, target_block):
        diff = target_block[0] - self.position[0], target_block[1] - self.position[1]
        
        move = None
        if diff[0] == 0 and diff[1] == 1:
            move = Move.Up
        elif diff[0] == 1 and diff[1] == 0:
            move = Move.Right
        elif diff[0] == 0 and diff[1] == -1:
            move = Move.Down
        elif diff[0] == -1 and diff[1] == 0:
            move = Move.Left
        else:
            raise Exception(f"Could not get move {(self.position)} -> {(target_block)}")
            
        return move.value
        


class Grid:
    def __init__(self):
        self.blocks = dict()
        self.blocks[(0, 0)] = Block.Open_Passage
    
    def get_path_to(self, target_position, source_position):
        reachable_positions = self._bfs(target_position, source_position)
        
        return self._build_path(target_position, source_position, reachable_positions)
        

    def _bfs(self, target_position, source_position):
        reachable_positions = dict()
        visited = set()
        queue = list()
        
        queue.append(source_position)
        
        while queue:
            current = queue.pop(0)
            
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    reachable_positions[neighbor] = current
                    
                    if neighbor == target_position:
                        return reachable_positions
                        
                    queue.append(neighbor)
                    
        raise Exception(f"Could not reach target position: {(source_position)} -> {(target_position)}")
        
    def _build_path(self, target_position, source_position, reachable_positions):
        path = list()
        current = target_position
        
        while current != source_position:
            position = current
            path.append(position)
            current = reachable_positions[current]

        path.append(source_position)
        path.reverse()

        return path
    
    def get_neighbors(self, position):
        for x_offset, y_offset in ((0, 1), (1, 0), (0, -1), (-1, 0)):
            canditdate_position = (position[0] + x_offset, position[1] + y_offset)

            if canditdate_position not in self.blocks:
                continue
            
            canditdate_block = self.blocks[canditdate_position]
            
            if canditdate_block == Block.Wall:
                continue
            
            yield canditdate_position

    
Finished = object()
Input = object()
Output = namedtuple('Output', ('status'))


def start_computer(sequence):
    index = 0
    relative_base = 0
    memory = defaultdict(int)
    
    for i, value in enumerate(sequence):
        memory[i] = value
        
    diag_nbr = None
    
    while True:
        opt_code = memory[index]
        opt_code_with_modes = str(opt_code).zfill(5)
        opt_code = int(opt_code_with_modes[-2:])
        modes = opt_code_with_modes[:-2]
        
        
        par_1_address = get_parameter_address(memory, modes[-1], index + 1, relative_base)
        par_2_address = get_parameter_address(memory, modes[-2], index + 2, relative_base)
        par_3_address = get_parameter_address(memory, modes[-3], index + 3, relative_base)
        
        par_1 = memory[par_1_address]
        par_2 = memory[par_2_address]
        
        if opt_code == 99:
            break
        elif opt_code == 3:
            target_address = par_1_address
            memory[target_address] = yield Input
            index += 2
        elif opt_code == 4:
            diag_nbr = par_1
            yield Output(diag_nbr)                
            index += 2
        elif opt_code == 1:
            target_address = par_3_address
            memory[target_address] = par_1 + par_2
            index += 4
        elif opt_code == 2:
            target_address = par_3_address
            memory[target_address] = par_1 * par_2
            index += 4
        elif opt_code == 5:
            if par_1 != 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 6:
            if par_1 == 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 7:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 < par_2 else 0
            index += 4
        elif opt_code == 8:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 == par_2 else 0
            index += 4
        elif opt_code == 9:
            relative_base += par_1
            index += 2
        else:
            print(f"Wrong code: {opt_code}")  
    
    yield Finished

            
def get_parameter_address(memory, mode, par_index, relative_base):
    if mode == "0":
        return memory[par_index]
    elif mode == "1":
        return par_index
    elif mode == "2":
        return relative_base + memory[par_index]
    else:
        print(f"[ERROR] Wrong mode code: {mode}")

In [82]:
print(f"Part 1 solution: {part_1_solution(raw_input_data)}")

Part 1 solution: 282


In [85]:
def part_2_solution(raw_input):
    puzzle_input = list(map(int, raw_input.split(',')))
    
    robot = Robot(puzzle_input)
    
    grid = build_grid(robot)
    
    visited = set()
    queue = list()
    oxygen_position = grid.blocks[Block.Oxygen_System]
    max_lvl = 0
    
    queue.append((oxygen_position, 0))
        
    while queue:
        current, lvl = queue.pop(0)
        max_lvl = max(max_lvl, lvl)
        
        for neighbor in grid.get_neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)

                queue.append((neighbor, lvl + 1))
    
    return max_lvl

In [86]:
print(f"Part 2 solution: {part_2_solution(raw_input_data)}")

Part 2 solution: 286
